<a href="https://colab.research.google.com/github/jackiekuen2/recommender-systems/blob/main/Kaggle_The_Movie_Dataset_Content_based_Recommender_in_plot_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credits
- https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a
- https://www.kaggle.com/general/74235

In [1]:
!pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [4]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

the-movies-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip \*.zip  && rm *.zip

In [6]:
import pandas as pd

In [7]:
# Load movie metadata
metadata = pd.read_csv('movies_metadata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
# Top 5 movies by Revnue
metadata.sort_values(by='revenue', ascending=False).head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
14551,False,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.071,/kmcqlZGaSh20zpTbuoF0Cdn07dT.jpg,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2.787965e+09,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.2,12114.0
26555,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.starwars.com/films/star-wars-episod...,140607,tt2488496,en,Star Wars: The Force Awakens,Thirty years after defeating the Galactic Empi...,31.626,/weUSwMdQIa3NaXVzwUoIIcAi85d.jpg,"[{'name': 'Lucasfilm', 'id': 1}, {'name': 'Tru...","[{'iso_3166_1': 'US', 'name': 'United States o...",2015-12-15,2.068224e+09,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every generation has a story.,Star Wars: The Force Awakens,False,7.5,7993.0
1639,False,NaN,200000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.titanicmovie.com,597,tt0120338,en,Titanic,"84 years later, a 101-year-old woman named Ros...",26.8891,/kHXEpyfl6zqn8a6YuozZUujufXf.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1997-11-18,1.845034e+09,194.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Nothing on Earth could come between them.,Titanic,False,7.5,7770.0
17818,False,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",http://marvel.com/avengers_movie/,24428,tt0848228,en,The Avengers,When an unexpected enemy emerges and threatens...,89.8876,/cezWGskPY5x7GaglTTRN4Fugfb8.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-04-25,1.519558e+09,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some assembly required.,The Avengers,False,7.4,12000.0
25084,False,"{'id': 328, 'name': 'Jurassic Park Collection'...",150000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.jurassicworld.com/,135397,tt0369610,en,Jurassic World,Twenty-two years after the events of Jurassic ...,32.7905,/jjBgi2r5cRt36xF6iNUEhzscEcb.jpg,"[{'name': 'Universal Studios', 'id': 13}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2015-06-09,1.513529e+09,124.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The park is open.,Jurassic World,False,6.5,8842.0


In [9]:
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

# II. Content-based Recommender in plot description
- Using plot description --> metadata['overview']
- compute pairwise cosine similarity scores for all movies based on their plot descriptions --> NLP problem
- to compute the word vectors of each overview or document --> This will give you a matrix where each column represents a word in the overview vocabulary (all the words that appear in at least one document), and each column represents a movie
- Cleasning before vectorization: 
    - Remove stop words like 'the', 'an', etc. since they do not give any useful information about the topic;
    - Replace not-a-number values with a blank string;

## TF-IDF Vectorizer
- down-weighted by the number of documents in which it occurs. This is done to reduce the importance of words that frequently occur in plot overviews

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
tfidf = TfidfVectorizer(stop_words='english')

# Fill Nan with empty
metadata['overview'] = metadata['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

In [12]:
tfidf_matrix.shape

(45466, 75827)

In [13]:
# List some feature names
tfidf.get_feature_names()[5000:5010]

['avails',
 'avaks',
 'avalanche',
 'avalanches',
 'avallone',
 'avalon',
 'avant',
 'avanthika',
 'avanti',
 'avaracious']

## Cosine Similarity using linear kernel
- Since you have used the TF-IDF vectorizer, calculating the dot product between each vector will directly give you the cosine similarity score. Therefore, you will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.
- This would return a matrix of shape 45466x45466, which means each movie overview cosine similarity score with every other movie overview. 

In [14]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the consine similarity matrix, by calculating the dot product of TF-IDF matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [15]:
cosine_sim.shape

(45466, 45466)

In [16]:
cosine_sim[1]

array([0.01504121, 1.        , 0.04681953, ..., 0.        , 0.02198641,
       0.00929411])

## Top 10 most similar movies

In [17]:
 # Assigne indices to movive title
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

In [18]:
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

In [19]:
# top 10 movies with similar descriptions
def get_recommendations(title, cosine_sim, topk=10):
    # Get the index of target movie
    idx = indices[title]
    # Get pairwise similariy scores of all movies that are similar to target movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Reversed-Sort the sim scores
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    # Get the top k, except the first one (the movie itself)
    sim_scores = sim_scores[1:topk]
    movie_indices = [i[0] for i in sim_scores]
    # return pd.DataFrame(metadata['title'].iloc[movie_indices])
    rec_results = pd.DataFrame(metadata['title'].iloc[movie_indices])
    rec_results['sim_scores'] = [i[1] for i in sim_scores]
    return rec_results

In [21]:
get_recommendations('Jurassic World', cosine_sim)

,title,sim_scores
1474,The Lost World: Jurassic Park,0.247255
13961,Dark Ride,0.195840
475,Jurassic Park,0.181525
30975,Jurassic City,0.170766
8757,Futureworld,0.169373
30718,It Waits,0.165788
24186,Trailer Park Boys: Don't Legalize It,0.156942
11644,The Grim Reaper,0.154276
31263,Tom and Jerry's Giant Adventure,0.152503


In [22]:
get_recommendations('The Dark Knight Rises', cosine_sim)

,title,sim_scores
12481,The Dark Knight,0.326639
150,Batman Forever,0.316211
1328,Batman Returns,0.296139
15511,Batman: Under the Red Hood,0.287983
585,Batman,0.270730
21194,Batman Unmasked: The Psychology of the Dark Kn...,0.253149
9230,Batman Beyond: Return of the Joker,0.234360
18035,Batman: Year One,0.204338
19792,"Batman: The Dark Knight Returns, Part 1",0.198813


In [23]:
get_recommendations('The Godfather', cosine_sim)

,title,sim_scores
1178,The Godfather: Part II,0.475954
44030,The Godfather Trilogy: 1972-1990,0.356142
1914,The Godfather: Part III,0.183049
23126,Blood Ties,0.153702
11297,Household Saints,0.140767
34717,Start Liquidation,0.134386
10821,Election,0.132326
38030,A Mother Should Be Loved,0.130311
17729,Short Sharp Shock,0.129697
